In [1]:
import numpy as np
import pandas as pd
import os 
import tensorflow as tf
import tensorflow.keras as keras
from keras import Model
from keras.applications.densenet import DenseNet121
from keras.applications import vgg16
from keras.preprocessing import image
from keras.applications.densenet import preprocess_input, decode_predictions
from keras.layers import GlobalMaxPooling2D
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pathlib
from sklearn.metrics.pairwise import linear_kernel
import tensorflow_hub as hub
from PIL import Image

In [2]:
path = 'archive/'
dataset_path = pathlib.Path(path)
dirs_names = os.listdir(dataset_path) # list content of dataset
dirs_names

styles_df = pd.read_csv(path + "styles.csv", nrows=20000, error_bad_lines=False) # Read n product and drop bad lines 
styles_df['image'] = styles_df.apply(lambda x: str(x['id']) + ".jpg", axis=1) # Make image column contains (id.jpg)
styles_df.head()

C:\Users\samis\AppData\Local\Temp\ipykernel_15340\1805511326.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  styles_df = pd.read_csv(path + "styles.csv", nrows=20000, error_bad_lines=False) # Read n product and drop bad lines
Skipping line 6044: expected 10 fields, saw 11
Skipping line 6569: expected 10 fields, saw 11
Skipping line 7399: expected 10 fields, saw 11
Skipping line 7939: expected 10 fields, saw 11
Skipping line 9026: expected 10 fields, saw 11
Skipping line 10264: expected 10 fields, saw 11
Skipping line 10427: expected 10 fields, saw 11
Skipping line 10905: expected 10 fields, saw 11
Skipping line 11373: expected 10 fields, saw 11
Skipping line 11945: expected 10 fields, saw 11
Skipping line 14112: expected 10 fields, saw 11
Skipping line 14532: expected 10 fields, saw 11
Skipping line 15076: expected 10 fields, saw 12



,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,53759.jpg


In [3]:
top_25 = styles_df['articleType'].value_counts().sort_values(ascending=False).head(25)
top_25_list = top_25.index
print(top_25_list)


# get only data with the more used 
styles_df_processed = styles_df.loc[styles_df['articleType'].isin(top_25_list), :]




Index(['Tshirts', 'Shirts', 'Casual Shoes', 'Watches', 'Sports Shoes',
       'Kurtas', 'Tops', 'Handbags', 'Heels', 'Sunglasses', 'Flip Flops',
       'Wallets', 'Sandals', 'Briefs', 'Belts', 'Backpacks', 'Socks',
       'Perfume and Body Mist', 'Formal Shoes', 'Jeans', 'Trousers', 'Shorts',
       'Flats', 'Bra', 'Sarees'],
      dtype='object')


In [4]:
#function for Preprocessing the data 

# Inputs must respect these conditions : 

   # be four dimensional Tensors of the shape (batch_size, height, width, num_channels). Note that the model expects images with channels_last property. num_channels must be 3.
   # be resized to 224x224 resolution.
   # have pixel values in the range [-1, 1].

def img_path(img):
    """ Take image name(id) and return the complete path of it """
    return path + 'images/' + img

def preprocess_image(image_path):
    # Load the image from file
    image = tf.io.read_file(image_path)
    
    # Decode the image and convert it to a tensor
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # Convert the data to a four-dimensional tensor
    image = tf.expand_dims(image, axis=0)
    
    # Resize the data to a resolution of 224x224
    image = tf.image.resize(image, (224, 224))
    
    # Normalize the pixel values to the range [-1, 1]
    image = (image / 127.5) - 1
    
    # Add a channels dimension to the data
    image = tf.transpose(image, (0, 3, 1, 2))
    
    return image


In [5]:
def load_labels_from_files(label_files):
    path = 'archive/'
    dataset_path = pathlib.Path(path)
    dirs_names = os.listdir(dataset_path) # list content of dataset
    # Create an empty list to store the labels
    labels = []

    # Iterate over the label files
    styles_df = pd.read_csv(path + "styles.csv", nrows=40000, error_bad_lines=False) # Read n product and drop bad lines 
    styles_df['image'] = styles_df.apply(lambda x: str(x['id']) + ".jpg", axis=1) # Make image column contains (id.jpg)

    return labels

def load_images_from_files(image_files):
  
    # Create an empty list to store the images
    images = []

    # Iterate over the image files
    for file in image_files:
        # Load the image using PIL
        try:
            
            image = preprocess_image(img_path(file))
            images.append(image)
        except FileNotFoundError:
            print("Could not find the specified file : ",path + 'images/')
            continue
        except ValueError:
            print("not enough row and columns")
            continue
        except Exception as e:
            print("An error occurred:", e)
        
        

    # Convert the list of images to a numpy array
    images = np.array(images)

    return images

images = load_images_from_files(styles_df_processed['image'].tolist())
labels = styles_df_processed['articleType'].tolist()


not enough row and columns
not enough row and columns


In [12]:
import torch
import torchvision

# Load the pretrained VGG16 model
model = torchvision.models.vgg16(pretrained=True)

# Freeze the parameters of the convolutional layers
for param in model.parameters():
    param.requires_grad = False

# Replace the fully connected layers with a new fully connected layer
# that has the same number of output units as the number of classes in the new dataset
num_classes = 10
model.classifier[6] = torch.nn.Linear(4096, num_classes)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and the optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Load the training and validation datasets
train_dataset = torchvision.datasets.ImageFolder(...)
val_dataset = torchvision.datasets.ImageFolder(...)

# Define the data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

# Fine-tune the model
for epoch in range(num_epochs):
    # Training
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        print("Epoch {}: Validation Accuracy = {}".format(epoch+1, val_accuracy))


<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x0000020B9D480C40>>


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(test_loss," ",test_acc)